# Introducción

“Nonlinear Conjugate Gradient for Smooth Convex Functions” de Sahar Karimi y Stephen Vavasis propone un método llamado C+AG (conjugate plus accelerated gradient) que intenta cerrar la brecha entre el método de gradiente conjugado no lineal (NCG), óptimo para funciones cuadráticas, y el método de gradiente acelerado (AG), óptimo para funciones convexas suaves.

El método de Gradiente Conjugados no lineales (NCG) es ampliamente utilizado para problemas de optimización sin restricciones, sin embargo, en los casos en que es aplicado a funciones convexas suaves sólo satisface límites de complejidad débiles.

Por otro lado, el método del Gradiente Acelerado de Nesterov (AG) es un método óptimo para esta clase de funciones (convexas suaves), pero no es tan eficiente como NCG para funciones cuadráticas:

* para el caso de las funciones cuadráticas, la decisión más apropiada sería utilizar un método de Gradiente Conjugado, por tanto es claro que existe una brecha en las opciones de los algoritmos de optimización disponibles.

* NCG es la solución óptima para funciones cuadráticas y de hecho demuestra un desempeño bastante apropiado para funciones generales, sin embargo tiene límites de complejidad bastante pobres comparados con AG.



## Gradiente Conjugado + Acelerado

El método de Gradiente Conjugado + Acelerado (C+AG) nace como una propuesta para cerrar esta brecha en nuestras opciones para la solución de los problemas anteriormente mencionados, busca ser óptimo para funciones cuadráticas además de satisfacer los mejores límites de complejidad para funciones convexas suaves más generales.

Idea:
* Seguir los pasos del método de Gradiente Conjugado (hasta que el progreso comience a ser insuficiente)
* Cambiar los pasos a los del método de Gradiente Acelerado
* Al final regresar al Gradiente Conjugado.



## Objetivos de la implementación

Implementar los métodos C+AG, AG, y NCG, y encontrar ciertos casos específicos en los cuáles el método de C+AG supera a ambos métodos AG y NCG.

# Fundamentos teóricos
El problema bajo consideración es minimizar una función L-Suave convexa $f: \mathcal{R}^{n} → \mathcal{R}$ sin restricciones.

* $\textbf{Definicion:}$ Una función es L-Convexa $\forall \; x,y \in \; \mathcal{R}^{n}$ si:

\begin{equation}
  f(y) - f(x) - \nabla f(x)^{T} (y-x) \leq L ||x-y||^{2}/2
\end{equation}

También se considerarán funciones l-fuertemente convexas.

* $\textbf{Definicion:}$ Una función es l-fuertemente convexa si $\forall \; x,y \in \; \mathcal{R}^{n}$ si:

\begin{equation}
  f(y) - f(x) - \nabla f(x)^{T} (y-x) \geq l ||x-y||^{2}/2
\end{equation}

Para **funciones L-convexas suaves** está probado que el método de gradiente acelerado requiere de $O(-\epsilon^{-1/2})$ iteraciones para alcanzar una precisión deseada de $\epsilon$.

Sin embargo, teóricamente NCG no tiene límites de complejidad óptimos para funciones convexas suaves generales, de modo que podría necesitar un mayor número de iteraciones que el método AG para este tipo de funciones.

Para **funciones l-fuertemente convexas** está probado que el método de gradiente acelerado requiere de $O(\sqrt{L/l} \: ln(1/\epsilon))$ iteraciones para alcanzar una precisión deseada de $\epsilon$.

De igual manera, teóricamente NCG no tiene límites de complejidad óptimos para esta clase de funciones, dando como resultado un posible peor desempeño que el método de AG.

Sin embargo, el método de C+AG propuesto por los autores cumple los límites de complejidad del método AG para **funciones L-convexas suaves** y para **funciones l-fuertemente** convexas, es decir $O(-\epsilon^{-1/2})$ y $O(\sqrt{L/l} \: ln(1/\epsilon))$, siendo una solución óptima para este tipo de problemas.

Además, mantiene la característica del método NCG de ser una solución óptima para las funciones cuadráticas, a diferencia del método AG. Por tanto vemos que el método propuesto C+AG combina los beneficios de ambos métodos manteniendo límites de complejidad óptimos para funciones generales y siendo también una solución óptima para funciones cuadráticas.



## ¿Cómo C+AG logra cerrar esta brecha?

1. **Transición dinámica entre NCG y AG:** C+AG inicia con los pasos del gradiente conjugado no lineal, los cuáles son óptimos para solucionar funciones cuadráticas, lo que provoca que para estos casos se haga un progreso rápido.
En el caso en que no se el progreso inicial no sea suficiente entonces se cambian los pasos a los de gradiente acelerado para asegurarse de mantener los límites de complejidad del mismo y reducir el número de iteraciones necesarias.



2. **Medida de progreso basada en Nesterov:** C+AG utiliza una medida de progreso basada en la secuencia de estimación de AG (Nesterov), con esto podemos evaluar si el progreso realizado es suficiente. Esta medida se basa en una secuencia de funciones cuadráticas fuertemente convexas $\phi_{k}(x)$, llamada secuencia de estimación.

  Primero inicializamos la secuencia:

  \begin{equation}
    \phi_{0}(x) = f(x_{0}) + \frac{L}{2}||x - v_{0}||^{2}
  \end{equation}

  donde $v_{0} = x_{0}$ y $L$ es la constante de suavidad de f.

  Después, para $k \geq 1$ la secuencia se actualiza de la siguiente forma:

  \begin{equation}
    \phi_{k+1}(x) = (1 - \theta_{k})\phi_{k}(x) + \theta_{k} \left[ f(\bar{x}_{k}) + \nabla f(\bar{x}_{k})^{T}(x - \bar{x}_{k}) + \frac{l}{2}||x - \bar{x}_{k}||^{2}\right]
  \end{equation}

  donde $\phi_{k} \in (0,1)$ se elige como la raíz positiva de la ecuación cuadrática:

  \begin{equation}
    L \theta_{k}^{2} + (\gamma_{k} - l)\theta_{k} - \gamma_{k} = 0
  \end{equation}

  donde $\gamma_{k}$ es un parámetro auxiliar que se actualiza cada iteración de la forma:

  \begin{equation}
    \gamma_{k+1} = (1- \theta_{k}) \gamma_{k} + \theta_{k} l
  \end{equation}

  Y $\bar{x}_{k}$ es el punto donde se evalúa el gradiente en la iteración k. En la mayoría de los casos $\bar{x}_{k}$ es simplemente el punto actual $x_{k}$.

  Después de calcular la secuencia, realizamos el test de progreso suficiente para determinar si el método debe continuar con gradiente conjugado o si debe cambiar a gradiente acelerado. El test se basa en verficar si el valor de la función objetivo $f(x_{k})$ es menor o igual al valor de la función $\phi_{k}(x_{k})$, si esta condición se cumple, entonces consideramos que ha hecho suficiente progreso:

  \begin{equation}
    f(x_{k}) \leq \phi_{k}^{*}
  \end{equation}

  Donde $\phi_{k}^{*}$ es el mínimo de la función $\phi_{k}(x)$



3. **Reinicio del gradiente conjugado:** En casos cuando el gradiente conjugado no haga suficiente progreso, el algoritmo utiliza un reinicio con un paso de descenso más pronunciado, con esto es seguro que el algoritmo no se estanque y siga avanzando.



4. **Evaluación adaptativa del tamaño de paso y los parámetros:** Con el método se incluye una función para estimar el parámetro L de forma adaptativa en caso de que este no se conozca a priori.

# Implementación de C+AG en python



In [ ]:
import numpy as np

def ComputeThetaGamma(L, ell, gamma_k):

    a = L
    b = gamma_k - ell
    c = -gamma_k
    theta_k = (-b + np.sqrt(b**2 - 4*a*c)) / (2*a)
    gamma_k_plus_1 = (1 - theta_k) * gamma_k + theta_k * ell

    return theta_k, gamma_k_plus_1

def EstimateL(f, x0, f0, grad, L):

    for _ in range(60):
        f1 = f(x0 - grad / L)
        if f1 >= f0 - np.dot(grad, grad) / (2 * L) and abs(f1 - f0) >= 1e-11 * abs(f0):
            L *= np.sqrt(2)
        else:
            return L

    raise ValueError("Line search failed to determine L; possible incorrect gradient function or excessive roundoff error")

def EstimateLInitial(f, x0, grad, L_init=1.0):

    L = L_init
    for _ in range(100):
        if f(x0 - grad / L) < f(x0) - np.dot(grad, grad) / (2 * L):
            L /= np.sqrt(2)
        else:
            return EstimateL(f, x0, f(x0), grad, L)

    raise ValueError("f may be unbounded below")

def ComputeVPhiStar(theta_k, gamma_k, gamma_k_plus_1, ell, v_k, phi_star_k, x_bar_k, f_x_bar_k, grad_x_bar_k):

    v_k_plus_1 = (1 / gamma_k_plus_1) * ((1 - theta_k) * gamma_k * v_k + theta_k * ell * x_bar_k - theta_k * grad_x_bar_k)

    phi_star_k_plus_1 = (1 - theta_k) * phi_star_k + theta_k * f_x_bar_k - (theta_k**2 / (2 * gamma_k_plus_1)) * np.dot(grad_x_bar_k, grad_x_bar_k) + \
                        (theta_k * (1 - theta_k) * gamma_k / gamma_k_plus_1) * (ell * np.dot(x_bar_k - v_k, x_bar_k - v_k) / 2 + np.dot(grad_x_bar_k, v_k - x_bar_k))

    return v_k_plus_1, phi_star_k_plus_1

def CPlusAG(f, grad, x0, ell=0, L=np.nan, gtol=1e-8):
    # Inicialización

    f0 = f(x0)
    grad0 = grad(x0)
    phi_star_0 = f0
    v0 = x0
    only_ag = False
    icg = 0
    iag = 0
    p0 = -grad0
    LNaNFlag = np.isnan(L)
    if LNaNFlag:
        L = EstimateLInitial(f, x0, grad0)
        ell = 0
    gamma0 = L
    xk = x0
    vk = v0
    phi_star_k = phi_star_0
    pk = p0

    # Ciclo de iteraciones
    for k in range(int(1e6)):
        theta_k, gamma_k_plus_1 = ComputeThetaGamma(L, ell, gamma0)
        for whichsteptype in range(1, 4):
            # Pasos de Gradiente Conjugado (GC)
            if whichsteptype <= 2:

                if only_ag:
                    continue

                if icg >= 6 * len(x0) + 1 or whichsteptype == 2:
                    pk = -grad(xk)
                    icg = 0

                if icg == 0 and k > 0 and LNaNFlag:
                    L = EstimateL(f, xk, f(xk), grad(xk), L)

                icg += 1
                iag = 0
                x_tilde = xk + pk / L
                grad_tilde = grad(x_tilde)

                if np.linalg.norm(grad_tilde) <= gtol:
                    return x_tilde, k, True

                Ap = L * (grad_tilde - grad(xk))
                pAp = np.dot(pk, Ap)

                if np.dot(grad(xk), pk) >= 0 or pAp <= 0:
                    continue

                alpha_k = -np.dot(grad(xk), pk) / pAp
                xk_plus_1 = xk + alpha_k * pk
                fk_plus_1 = f(xk_plus_1)
                gradk_plus_1 = grad(xk_plus_1)

                if np.linalg.norm(gradk_plus_1) <= gtol:
                    return xk_plus_1, k, True
                vk_plus_1, phi_star_k_plus_1 = ComputeVPhiStar(theta_k, gamma0, gamma_k_plus_1, ell, vk, phi_star_k, xk, f(xk), grad(xk))

                if fk_plus_1 <= phi_star_k_plus_1:
                    y_hat = gradk_plus_1 - grad(xk)
                    beta_1 = np.dot((y_hat - pk * 2 * np.dot(y_hat, y_hat) / np.dot(y_hat, pk)), gradk_plus_1) / np.dot(y_hat, pk)
                    beta_2 = -1 / np.linalg.norm(pk) * min(0.01 * np.linalg.norm(grad0), np.linalg.norm(gradk_plus_1))
                    beta_k_plus_1 = max(beta_1, beta_2)
                    pk_plus_1 = -gradk_plus_1 + beta_k_plus_1 * pk
                    pk = pk_plus_1
                    xk = xk_plus_1
                    vk = vk_plus_1
                    phi_star_k = phi_star_k_plus_1
                    break
            # Pasos de Gradiente Acelerado (AG)
            else:
                if not only_ag:
                    only_ag = True
                    iag = 0
                    icg = 0

                iag += 1
                x_bar_k = (theta_k * gamma0 * vk + gamma_k_plus_1 * xk) / (gamma0 + theta_k * ell)
                grad_bar_k = grad(x_bar_k)

                if np.linalg.norm(grad_bar_k) <= gtol:
                    return x_bar_k, k, True

                if LNaNFlag:
                    L = EstimateL(f, xk, f(xk), grad(xk), L)

                xk_plus_1 = x_bar_k - grad_bar_k / L
                vk_plus_1, phi_star_k_plus_1 = ComputeVPhiStar(theta_k, gamma0, gamma_k_plus_1, ell, vk, phi_star_k, x_bar_k, f(x_bar_k), grad_bar_k)

                if iag % 8 == 0 and f(xk_plus_1) <= f(x_bar_k) - (4 / 5) * np.dot(grad_bar_k, grad_bar_k + grad(xk_plus_1)) / (2 * L):
                    fk_plus_1 = f(xk_plus_1)
                    gradk_plus_1 = grad(xk_plus_1)
                    pk = -gradk_plus_1
                    xk = xk_plus_1
                    vk = vk_plus_1
                    phi_star_k = phi_star_k_plus_1
                    only_ag = False
                    break

        gamma0 = gamma_k_plus_1

    return xk, k, False


# Implementación de Gradiente Conjugado

In [ ]:
def backtracking(alpha_ini, rho, c1, c2, xk, f, grad_f, pk, Nb):

    alpha = alpha_ini

    for i in range(Nb):
        if f(xk + alpha*pk) <= f(xk) + c1*alpha*np.dot(grad_f(xk), pk) and np.dot(grad_f(xk + alpha*pk), pk) >= c2*(np.dot(grad_f(xk), pk)):
            return alpha
        alpha = rho*alpha

    return alpha



def GC_no_lineal(f, grad_f, x0, tol, N, alpha_ini, rho, c1, c2, Nb):

    nr = 0
    g0 = grad_f(x0)
    d0 = -g0

    gk = g0
    xk = x0
    dk = d0

    for k in range(N):
        xk_prev = xk
        gk_prev = gk
        if np.linalg.norm(gk) < tol:
            return xk, gk, k, nr, True

        alpha = backtracking(alpha_ini, rho, c1, c2, xk, f, grad_f, dk, Nb)
        xk = xk_prev + alpha*dk
        fk = f(xk)
        gk = grad_f(xk)
        yk = gk - gk_prev
        if np.linalg.norm(np.dot(gk, gk_prev)) < 0.2*((np.linalg.norm(gk))**2):
            beta = np.dot(gk, gk)/np.dot(gk_prev, gk_prev)
        else:
            beta = 0
            nr = nr +1
        dk = -gk + beta*dk

    return xk, gk, k, nr, False

# Implementación de Gradiente Acelerado (Nesterov)

In [ ]:
def N_AG(f, grad_f, x0, tol, N, alpha_ini, gamma):
    xk = x0
    yk = x0
    vk = np.zeros_like(x0)

    for k in range(N):
        gk = grad_f(yk)

        vk = gamma * vk + alpha_ini * gk

        if np.any(np.isnan(vk)):
            print(f"NaN encontrado en vk en la iteración {k}")
            break

        xk_prev = xk
        xk = yk - vk
        yk = xk + gamma * (xk - xk_prev)

        if np.linalg.norm(gk) < tol:
            return xk, gk, k, True

    return xk, gk, N, False

# Prueba 1:

$f(x) = \frac{1}{2} x^{T} A_{1} - b_{1}^{T} x$

 Donde A1 = nI + 1, siendo n el tamaño de x0, I la matriz identidad, 1 una matriz de 1's, y b un vector unitario

In [ ]:
def cuadratica_1_1000(x):
    I = np.eye(1000)
    uno = np.ones((1000,1000))
    A1 = 1000 * I + uno
    b1 = np.ones(1000)
    return 0.5 * np.dot(x.T, np.dot(A1, x)) - np.dot(b1.T, x)

def grad_cuadratica_1_1000(x):
    I = np.eye(1000)
    uno = np.ones((1000,1000))
    b1 = np.ones(1000)
    A1 = 1000 * I + uno
    return 0.5*(np.dot(A1.T, x) + np.dot(A1,x)) - b1

# Solución 1 con GC

In [ ]:
import sys
# Resolvemos para la funcion cuadratica en R10

n = 1000
N = 5000
rho = 0.5
c1 = 0.001
c2 = 0.01
Nb = 500
alpha_ini = 1
x0 = np.zeros(1000)
epsilon = sys.float_info.epsilon
tol = np.sqrt(n)*(epsilon**(1/3))

xk, gk, k, nr, bres, = GC_no_lineal(cuadratica_1_1000, grad_cuadratica_1_1000, x0, tol, N, alpha_ini, rho, c1, c2, Nb)

print("Dimension: ", n)
print("Número de iteraciones realizadas: ", k)

for i in range(4):
    print("Entrada ", i, " de xk: ", xk[i])

for i in range(3, -1, -1):
    print("Entrada ", n - i, "de xk: ", xk[i])

print("¿Se cumplió la condición de paro?", bres)
print("Número de reinicios nr: ", nr)

Dimension:  1000
f(x0) =  0.0
Número de iteraciones realizadas:  251
f(xk) =  -0.24999999999146527
Entrada  0  de xk:  0.0005000029213602938
Entrada  1  de xk:  0.0005000029213602938
Entrada  2  de xk:  0.0005000029213602938
Entrada  3  de xk:  0.0005000029213602938
Entrada  997 de xk:  0.0005000029213602938
Entrada  998 de xk:  0.0005000029213602938
Entrada  999 de xk:  0.0005000029213602938
Entrada  1000 de xk:  0.0005000029213602938
Norma de gk:  0.00018476304800255685
¿Se cumplió la condición de paro? True
Número de reinicios nr:  251


# Solución 1 con AG (Nesterov):

In [ ]:
import sys
# Resolvemos para la funcion cuadratica en R10

n = 1000
N = 5000
gamma = 0.5
alpha_ini = 0.0001
x0 = np.zeros(1000)
epsilon = sys.float_info.epsilon
tol = np.sqrt(n)*(epsilon**(1/3))
# Ejecutar el método de Nesterov
x_opt, grad_opt, iter, bres = N_AG(cuadratica_1_1000, grad_cuadratica_1_1000, x0, tol, N, alpha_ini, gamma)

print("Dimension: ", n)
print("\n")

for i in range(4):
    print("Entrada ", i, " de xk: ", x_opt[i])

for i in range(3, -1, -1):
    print("Entrada ", n - i, "de xk: ", x_opt[i])

print("\n")
print(f"Iteraciones realizadas: {iter}")
print(f"¿Se cumplió la condición de paro?: {bres}")

Dimension:  1000


Entrada  0  de xk:  0.0005000052710371406
Entrada  1  de xk:  0.0005000052710371406
Entrada  2  de xk:  0.0005000052710371406
Entrada  3  de xk:  0.0005000052710371406
Entrada  997 de xk:  0.0005000052710371406
Entrada  998 de xk:  0.0005000052710371406
Entrada  999 de xk:  0.0005000052710371406
Entrada  1000 de xk:  0.0005000052710371406


Norma de gk: 5.801187865114948e-05
Iteraciones realizadas: 44
¿Se cumplió la condición de paro?: True


# Solución 1 con G+AC

In [ ]:
n = 1000
x0 = np.zeros(n)
L = float('nan')  # Valor inicial para L
l = 0
gtol = 1e-6


# Llamada a la función de optimización
x_opt, iter, bres = CPlusAG(cuadratica_1_1000, grad_cuadratica_1_1000, x0, l, L, gtol)

print("Dimension: ", n)
print("\n")

for i in range(4):
    print("Entrada ", i, " de xk: ", x_opt[i])

for i in range(3, -1, -1):
    print("Entrada ", n - i, "de xk: ", x_opt[i])

# Número de iteraciones realizadas
print('Iteraciones realizadas: ', iter)
print(f"¿Se cumplió la condición de paro?: {bres}")

Dimension:  1000


Entrada  0  de xk:  0.0005
Entrada  1  de xk:  0.0005
Entrada  2  de xk:  0.0005
Entrada  3  de xk:  0.0005
Entrada  997 de xk:  0.0005
Entrada  998 de xk:  0.0005
Entrada  999 de xk:  0.0005
Entrada  1000 de xk:  0.0005
Iteraciones realizadas:  0
¿Se cumplió la condición de paro?: True


# Prueba 2:

$f(x) = \frac{1}{2} x^{T} A_{2} - b_{2}^{T} x$

 donde $a_{ij} = exp(-0.25(i-j)^{2})$ y b es un vector unitario

In [ ]:
def cuadratica_2_10(x):
  A2 = np.empty([10,10], dtype = float)
  for i in range(10):
    for j in range(10):
      A2[i][j] = np.exp(-0.25*(i-j)**2)
  b2 = np.ones(10)
  return 0.5 * np.dot(x.T, np.dot(A2, x)) - np.dot(b2.T, x)

def grad_cuadratica_2_10(x):
  A2 = np.empty([10,10], dtype = float)
  b2 = np.ones(10)
  for i in range(10):
    for j in range(10):
      A2[i][j] = np.exp(-0.25*(i-j)**2)
  return 0.5*(np.dot(A2.T, x) + np.dot(A2,x)) - b2

# Solución 2 con GC

In [ ]:
import sys
# Resolvemos para la funcion cuadratica en R10

n = 10
N = 5000
rho = 0.5
c1 = 0.001
c2 = 0.01
Nb = 500
alpha_ini = 1
x0 = np.zeros(10)
epsilon = sys.float_info.epsilon
tol = np.sqrt(n)*(epsilon**(1/3))

xk, gk, k, nr, bres, = GC_no_lineal(cuadratica_2_10, grad_cuadratica_2_10, x0, tol, N, alpha_ini, rho, c1, c2, Nb)

print("Dimension: ", n)
print("Número de iteraciones realizadas: ", k)

for i in range(4):
    print("Entrada ", i, " de xk: ", xk[i])

for i in range(3, -1, -1):
    print("Entrada ", n - i, "de xk: ", xk[i])

print("¿Se cumplió la condición de paro?", bres)
print("Número de reinicios nr: ", nr)

Dimension:  10
Número de iteraciones realizadas:  1571
Entrada  0  de xk:  1.3688956634168
Entrada  1  de xk:  -1.165862919253239
Entrada  2  de xk:  1.6083890463638981
Entrada  3  de xk:  -0.6127911466676736
Entrada  7 de xk:  -0.6127911466676736
Entrada  8 de xk:  1.6083890463638981
Entrada  9 de xk:  -1.165862919253239
Entrada  10 de xk:  1.3688956634168
¿Se cumplió la condición de paro? True
Número de reinicios nr:  1230


# Solución 2 con AG (Nesterov):

In [ ]:
import sys
# Resolvemos para la funcion cuadratica en R10

n = 10
N = 5000
gamma = 0.5
alpha_ini = 0.1
x0 = np.zeros(10)
epsilon = sys.float_info.epsilon
tol = np.sqrt(n)*(epsilon**(1/3))
# Ejecutar el método de Nesterov
x_opt, grad_opt, iter, bres = N_AG(cuadratica_2_10, grad_cuadratica_2_10, x0, tol, N, alpha_ini, gamma)

print("Dimension: ", n)
print("\n")

for i in range(4):
    print("Entrada ", i, " de xk: ", x_opt[i])

for i in range(3, -1, -1):
    print("Entrada ", n - i, "de xk: ", x_opt[i])

print("\n")
print(f"Iteraciones realizadas: {iter}")
print(f"¿Se cumplió la condición de paro?: {bres}")

Dimension:  10


Entrada  0  de xk:  1.3688281296820384
Entrada  1  de xk:  -1.1656955388134709
Entrada  2  de xk:  1.6081598371057975
Entrada  3  de xk:  -0.6125963884633056
Entrada  7 de xk:  -0.6125963884633056
Entrada  8 de xk:  1.6081598371057975
Entrada  9 de xk:  -1.1656955388134709
Entrada  10 de xk:  1.3688281296820384


Iteraciones realizadas: 1967
¿Se cumplió la condición de paro?: True


# Solución 2 con G+AC

In [ ]:
n = 10
x0 = np.zeros(n)
L = float('nan')  # Valor inicial para L
l = 0
gtol = 1e-6


# Llamada a la función de optimización
x_opt, iter, bres = CPlusAG(cuadratica_2_10, grad_cuadratica_2_10, x0, l, L, gtol)

print("Dimension: ", n)
print("\n")

for i in range(4):
    print("Entrada ", i, " de xk: ", x_opt[i])

for i in range(3, -1, -1):
    print("Entrada ", n - i, "de xk: ", x_opt[i])

# Número de iteraciones realizadas
print('Iteraciones realizadas: ', iter)
print(f"¿Se cumplió la condición de paro?: {bres}")

Dimension:  10


Entrada  0  de xk:  1.3690991585450476
Entrada  1  de xk:  -1.1663768171766733
Entrada  2  de xk:  1.6090828050261778
Entrada  3  de xk:  -0.6133905258822363
Entrada  7 de xk:  -0.6133905258822363
Entrada  8 de xk:  1.6090828050261778
Entrada  9 de xk:  -1.1663768171766733
Entrada  10 de xk:  1.3690991585450476
Iteraciones realizadas:  4
¿Se cumplió la condición de paro?: True


# Prueba 3: Basis Pursuit Denoising

$min \frac{1}{2} ||Ax - b||^{2} + \lambda \sum_{i=1}^{n} \sqrt{x_{i}^{2} + \delta}$

In [ ]:
def abpdn_function(x, A, b, lambd, delta):
    Ax_minus_b = A @ x - b
    smooth_l1_term = np.sum(np.sqrt(x**2 + delta))
    return 0.5 * np.dot(Ax_minus_b, Ax_minus_b) + lambd * smooth_l1_term

def abpdn_gradient(x, A, b, lambd, delta):
    Ax_minus_b = A @ x - b
    smooth_l1_grad = x / np.sqrt(x**2 + delta)
    return A.T @ Ax_minus_b + lambd * smooth_l1_grad

# Función que genera la matriz utilizada por ABPDN
def create_abpdn_matrices(n, m, delta):
    A = np.diag(np.concatenate([np.ones(m), np.ones(n - m) * 1000]))
    b = np.sin(np.arange(1, n + 1))
    return A, b

# Asignamos los parámetros a la función ABPDN y creamos una función auxiliar para llamar usando solo x
n = 1000
m = 500
delta = 1e-5
lambd = 1e-3
A, b = create_abpdn_matrices(n, m, delta)

def test_abpdn_function(x):
    return abpdn_function(x, A, b, lambd, delta)

def test_abpdn_gradient(x):
    return abpdn_gradient(x, A, b, lambd, delta)

# Solución 3 con GC:

In [ ]:
n = 1000
N = 5000
rho = 0.5
c1 = 0.001
c2 = 0.01
Nb = 500
alpha_ini = 1
x0 = np.zeros(1000)
epsilon = sys.float_info.epsilon
tol = np.sqrt(n)*(epsilon**(1/3))

xk, gk, k, nr, bres, = GC_no_lineal(test_abpdn_function, test_abpdn_gradient, x0, tol, N, alpha_ini, rho, c1, c2, Nb)

print("Dimension: ", n)
print("Número de iteraciones realizadas: ", k)

for i in range(4):
    print("Entrada ", i, " de xk: ", xk[i])

for i in range(3, -1, -1):
    print("Entrada ", n - i, "de xk: ", xk[i])

print("¿Se cumplió la condición de paro?", bres)
print("Número de reinicios nr: ", nr)

Dimension:  1000
Número de iteraciones realizadas:  1312
Entrada  0  de xk:  0.8404707675021708
Entrada  1  de xk:  0.9082971904525491
Entrada  2  de xk:  0.14012022462607937
Entrada  3  de xk:  -0.7558023022086048
Entrada  997 de xk:  -0.7558023022086048
Entrada  998 de xk:  0.14012022462607937
Entrada  999 de xk:  0.9082971904525491
Entrada  1000 de xk:  0.8404707675021708
¿Se cumplió la condición de paro? True
Número de reinicios nr:  1274


# Solución 3 con AG (Nesterov):

In [ ]:
import sys
# Resolvemos para la funcion cuadratica en R10

n = 1000
N = 5000
gamma = 0.5
alpha_ini = 0.1
x0 = np.zeros(1000)
epsilon = sys.float_info.epsilon
tol = np.sqrt(n)*(epsilon**(1/3))
# Ejecutar el método de Nesterov
x_opt, grad_opt, iter, bres = N_AG(test_abpdn_function, test_abpdn_gradient, x0, tol, N, alpha_ini, gamma)

print("Dimension: ", n)
print("\n")

for i in range(4):
    print("Entrada ", i, " de xk: ", x_opt[i])

for i in range(3, -1, -1):
    print("Entrada ", n - i, "de xk: ", x_opt[i])

print("\n")
print(f"Iteraciones realizadas: {iter}")
print(f"¿Se cumplió la condición de paro?: {bres}")

NaN encontrado en vk en la iteración 60
Dimension:  1000


Entrada  0  de xk:  0.8404711418946417
Entrada  1  de xk:  0.9082975950173724
Entrada  2  de xk:  0.1401202874572304
Entrada  3  de xk:  -0.7558026389365773
Entrada  997 de xk:  -0.7558026389365773
Entrada  998 de xk:  0.1401202874572304
Entrada  999 de xk:  0.9082975950173724
Entrada  1000 de xk:  0.8404711418946417


Iteraciones realizadas: 5000
¿Se cumplió la condición de paro?: False


<ipython-input-17-62c4d16a9cca>:8: RuntimeWarning: overflow encountered in square
  smooth_l1_grad = x / np.sqrt(x**2 + delta)
<ipython-input-17-62c4d16a9cca>:7: RuntimeWarning: invalid value encountered in matmul
  Ax_minus_b = A @ x - b
<ipython-input-17-62c4d16a9cca>:8: RuntimeWarning: invalid value encountered in divide
  smooth_l1_grad = x / np.sqrt(x**2 + delta)


# Solución 3 con C+AG:

In [ ]:
n = 1000
x0 = np.zeros(n)
L = float('nan')  # Valor inicial para L
l = 0
gtol = 1e-6


# Llamada a la función de optimización
x_opt, iter, bres = CPlusAG(test_abpdn_function, test_abpdn_gradient, x0, l, L, gtol)

print("Dimension: ", n)
print("\n")

for i in range(4):
    print("Entrada ", i, " de xk: ", x_opt[i])

for i in range(3, -1, -1):
    print("Entrada ", n - i, "de xk: ", x_opt[i])

# Número de iteraciones realizadas
print('Iteraciones realizadas: ', iter)
print(f"¿Se cumplió la condición de paro?: {bres}")

Dimension:  1000


Entrada  0  de xk:  0.8404709799544308
Entrada  1  de xk:  0.9082974204823013
Entrada  2  de xk:  0.14012025557811464
Entrada  3  de xk:  -0.7558024927186807
Entrada  997 de xk:  -0.7558024927186807
Entrada  998 de xk:  0.14012025557811464
Entrada  999 de xk:  0.9082974204823013
Entrada  1000 de xk:  0.8404709799544308
Iteraciones realizadas:  10
¿Se cumplió la condición de paro?: True


# Prueba 4: Función de perdida logística

 $R(Ax) + \lambda ||x||^{2}/2 $

donde $R(v) = \sum_{i=1}^{m} r(v_{i})$

y $r(v_{i}) = ln(1 + e^{-v})$

In [ ]:
# Función logística
def R(v):
    return np.sum(np.log(1 + np.exp(-v)))

# Función de pérdida logística regularizada
def logistic_loss_function(x, A, lambd):
    return R(A @ x) + (lambd / 2) * np.linalg.norm(x) ** 2

# Gradiente de la función logística
def grad_R(v):
    return -1 / (1 + np.exp(v))

# Gradiente de la función de pérdida logística regularizada
def logistic_loss_grad(x, A, lambd):
    return A.T @ grad_R(A @ x) + lambd * x


# Definimos el tamaño de la matriz
m = 100
n = 50

# Fijamos la semilla del generador de números aleatorios para reproducibilidad
np.random.seed(0)

# Vector de unos
e = np.ones(n)

# Generamos la matriz A
sigma = 0.4
A = np.zeros((m, n))
for i in range(m):
    z_i = np.random.normal(0, sigma, n)
    A[i, :] = e / np.sqrt(n) + z_i

# Parámetros de regularización
lambd1 = 1e-4

def test_logloss_function(x):
  return logistic_loss_function(x, A, lambd)

def test_logloss_grad(x):
  return logistic_loss_grad(x, A, lambd)

# Solución 4 con GC:

In [ ]:
N = 5000
rho = 0.5
c1 = 0.001
c2 = 0.01
Nb = 500
alpha_ini = 1
x0 = np.zeros(50)
epsilon = sys.float_info.epsilon
tol = np.sqrt(n)*(epsilon**(1/3))

xk, gk, k, nr, bres, = GC_no_lineal(test_logloss_function, test_logloss_grad, x0, tol, N, alpha_ini, rho, c1, c2, Nb)

print("Dimension: ", n)
print("Número de iteraciones realizadas: ", k)

for i in range(4):
    print("Entrada ", i, " de xk: ", xk[i])

for i in range(3, -1, -1):
    print("Entrada ", n - i, "de xk: ", xk[i])

print("¿Se cumplió la condición de paro?", bres)
print("Número de reinicios nr: ", nr)

Dimension:  50
Número de iteraciones realizadas:  4999
Entrada  0  de xk:  2.2988868051911586
Entrada  1  de xk:  6.478331320454311
Entrada  2  de xk:  8.669251539813537
Entrada  3  de xk:  8.064267655852795
Entrada  47 de xk:  8.064267655852795
Entrada  48 de xk:  8.669251539813537
Entrada  49 de xk:  6.478331320454311
Entrada  50 de xk:  2.2988868051911586
¿Se cumplió la condición de paro? False
Número de reinicios nr:  5000


# Solución 4 con AG (Nesterov):

In [ ]:
import sys
# Resolvemos para la funcion cuadratica en R10

N = 50000
gamma = 0.5
alpha_ini = 1
x0 = np.zeros(n)
epsilon = sys.float_info.epsilon
tol = np.sqrt(n)*(epsilon**(1/3))
# Ejecutar el método de Nesterov
x_opt, grad_opt, iter, bres = N_AG(test_logloss_function, test_logloss_grad, x0, tol, N, alpha_ini, gamma)

print("Dimension: ", n)
print("\n")

for i in range(4):
    print("Entrada ", i, " de xk: ", x_opt[i])

for i in range(3, -1, -1):
    print("Entrada ", n - i, "de xk: ", x_opt[i])

print("\n")
print(f"Iteraciones realizadas: {iter}")
print(f"¿Se cumplió la condición de paro?: {bres}")

Dimension:  50


Entrada  0  de xk:  -1.5365069011796482
Entrada  1  de xk:  0.8977476876601626
Entrada  2  de xk:  2.562964241289959
Entrada  3  de xk:  2.7041663813147796
Entrada  47 de xk:  2.7041663813147796
Entrada  48 de xk:  2.562964241289959
Entrada  49 de xk:  0.8977476876601626
Entrada  50 de xk:  -1.5365069011796482


Iteraciones realizadas: 11758
¿Se cumplió la condición de paro?: True


# Solución 4 con C+AG:

In [ ]:
x0 = np.zeros(n)
L = float('nan')  # Valor inicial para L
l = 0
gtol = 1e-6

# Llamada a la función de optimización
x_opt, iter, bres = CPlusAG(test_logloss_function, test_logloss_grad, x0, l, L, gtol)

print("Dimension: ", n)
print("\n")

for i in range(4):
    print("Entrada ", i, " de xk: ", x_opt[i])

for i in range(3, -1, -1):
    print("Entrada ", n - i, "de xk: ", x_opt[i])

# Número de iteraciones realizadas
print('Iteraciones realizadas: ', iter)
print(f"¿Se cumplió la condición de paro?: {bres}")

Dimension:  50


Entrada  0  de xk:  -1.608055487676299
Entrada  1  de xk:  0.8901739574556402
Entrada  2  de xk:  2.5841621945415425
Entrada  3  de xk:  2.648803954952745
Entrada  47 de xk:  2.648803954952745
Entrada  48 de xk:  2.5841621945415425
Entrada  49 de xk:  0.8901739574556402
Entrada  50 de xk:  -1.608055487676299
Iteraciones realizadas:  24
¿Se cumplió la condición de paro?: True


# Prueba 5: Regresión de Huber

  $ \zeta(t) = \begin{cases}
      - \tau^{2} - \tau t & si \; t \leq \tau \\
      t^{2} & si \; t \in [-\tau, \tau]  \\
      -\tau^{2} + 2 \tau t &  si \; t \geq \tau
   \end{cases}$

In [ ]:
# Definimos la función de la regresión de Huber y su gradiente
def huber_regression_function(x, A, b, tau):
    Ax_minus_b = A @ x - b
    zeta = np.where(np.abs(Ax_minus_b) <= tau,
                    Ax_minus_b**2,
                    tau * (2 * np.abs(Ax_minus_b) - tau))
    return np.sum(zeta)

def huber_regression_gradient(x, A, b, tau):
    Ax_minus_b = A @ x - b
    grad_zeta = np.where(np.abs(Ax_minus_b) <= tau,
                         2 * Ax_minus_b,
                         tau * np.sign(Ax_minus_b))
    return A.T @ grad_zeta

# Función para crear la matriz A y el vector b para la función de HUber
def create_huber_matrices(n):
    A = np.eye(n + 1) - np.eye(n + 1, k=-1)
    b = np.ones(n + 1)
    b[-1] = -1.1 * n
    return A, b

n = 100
tau = 250
A, b = create_huber_matrices(n)

def test_huber_regression_function(x):
    return huber_regression_function(x, A, b, tau)

def test_huber_regression_gradient(x):
    return huber_regression_gradient(x, A, b, tau)


# Solución 5 con GC:

In [ ]:
N = 50001
rho = 0.5
c1 = 0.001
c2 = 0.1
Nb = 500
alpha_ini = 1
x0 = np.zeros(n + 1)
epsilon = sys.float_info.epsilon
tol = np.sqrt(n)*(epsilon**(1/3))

xk, gk, k, nr, bres, = GC_no_lineal(test_huber_regression_function, test_huber_regression_gradient, x0, tol, N, alpha_ini, rho, c1, c2, Nb)

print("Dimension: ", n)
print("Número de iteraciones realizadas: ", k)

for i in range(4):
    print("Entrada ", i, " de xk: ", xk[i])

for i in range(3, -1, -1):
    print("Entrada ", n - i, "de xk: ", xk[i])

print("¿Se cumplió la condición de paro?", bres)
print("Número de reinicios nr: ", nr)

Dimension:  100
Número de iteraciones realizadas:  50000
Entrada  0  de xk:  0.9924884935265623
Entrada  1  de xk:  1.9849690980029322
Entrada  2  de xk:  2.977472669103885
Entrada  3  de xk:  3.9699526084454906
Entrada  97 de xk:  3.9699526084454906
Entrada  98 de xk:  2.977472669103885
Entrada  99 de xk:  1.9849690980029322
Entrada  100 de xk:  0.9924884935265623
¿Se cumplió la condición de paro? False
Número de reinicios nr:  42620


# Solución 5 con AG (Nesterov):

In [ ]:
import sys
# Resolvemos para la funcion cuadratica en R10

N = 50000
gamma = 0.5
alpha_ini = 0.1
x0 = np.zeros(n + 1)
epsilon = sys.float_info.epsilon
tol = np.sqrt(n)*(epsilon**(1/3))
# Ejecutar el método de Nesterov
x_opt, grad_opt, iter, bres = N_AG(test_huber_regression_function, test_huber_regression_gradient, x0, tol, N, alpha_ini, gamma)

print("Dimension: ", n)
print("\n")

for i in range(4):
    print("Entrada ", i, " de xk: ", x_opt[i])

for i in range(3, -1, -1):
    print("Entrada ", n - i, "de xk: ", x_opt[i])

print("\n")
print(f"Iteraciones realizadas: {iter}")
print(f"¿Se cumplió la condición de paro?: {bres}")

Dimension:  100


Entrada  0  de xk:  0.9997254407063478
Entrada  1  de xk:  1.9994509471686597
Entrada  2  de xk:  2.9991765851271515
Entrada  3  de xk:  3.9989024202905465
Entrada  97 de xk:  3.9989024202905465
Entrada  98 de xk:  2.9991765851271515
Entrada  99 de xk:  1.9994509471686597
Entrada  100 de xk:  0.9997254407063478


Iteraciones realizadas: 43904
¿Se cumplió la condición de paro?: True


# Solución 5 con C+AG:

In [ ]:
x0 = np.zeros(n + 1)

x_opt, iter, bres = CPlusAG(test_huber_regression_function, test_huber_regression_gradient, x0)

print("Dimension: ", n)
print("\n")

for i in range(4):
    print("Entrada ", i, " de xk: ", x_opt[i])

for i in range(3, -1, -1):
    print("Entrada ", n - i, "de xk: ", x_opt[i])

# Número de iteraciones realizadas
print('Iteraciones realizadas: ', iter)
print(f"¿Se cumplió la condición de paro?: {bres}")

Dimension:  100


Entrada  0  de xk:  0.9999999999998714
Entrada  1  de xk:  1.9999999999997242
Entrada  2  de xk:  3.0000000000003957
Entrada  3  de xk:  3.9999999999996247
Entrada  97 de xk:  3.9999999999996247
Entrada  98 de xk:  3.0000000000003957
Entrada  99 de xk:  1.9999999999997242
Entrada  100 de xk:  0.9999999999998714
Iteraciones realizadas:  100
¿Se cumplió la condición de paro?: True


# Resumen de resultados


| Función               | Gradiente Conjugado | Gradiente Acelerado | C+AG |
|-----------------------|---------------------|---------------------|------|
| Cuadrática 1             | 251 (T)            | 44 (T)             | 0 (T) |
| Cuadrática 2             | 1571 (T)           | 1967  (T)           | 4 (T) |
| ABPD                  | 1312 (T)            | 5000 (F)            | 10 (T)|
| Logistic Loss (LL)    | 5000 (F)            | 11758 (T)            | 24 (T)|
| Huber Regression (HR) | 50000 (F)            | 43904 (T)            | 100 (T) |


# Conclusiones

Podemos ver que para las 5 funciones de prueba aplicadas, el método C+AG del artículo supera considerablemente a los métodos de CG y AG en número de iteraciones necesarias para alcanzar el mínimo de la función, cumpliendo nuestro objetivo al implementar el método de la publicación.

Aunque los métodos de CG y AC no convergieron para todas las pruebas, en realidad solo en la prueba de la función de perdida logística, el método de CG no se acercó lo suficiente al mínimo de la función (significando que requería un considerable número mayor de iteraciones, pero se decidió dejar así por cuestiones de economizar tiempo), en los demás casos CG para HR y AG para ABPD, los mínimos obtenidos en la última iteración fueron bastante cercanos al mínimo de la función, significando que solo requerían un poco más de iteraciones para alcanzar el valor buscado.

En cuánto a discrepancias con los resultados del artículo, solo tenemos una, que es para el caso de la función de perdida logística, en los resultados del artículo converge más rápido para CG que para C+AG, sin embargo en nuestra implementación ocurrió lo contrario, esto puede deberse a las diferencias en la implementación del método de CG, siendo la función utilizada por los autores del artículo una versión más sofisticada del método, podría ser interesante repetir estas pruebas con una versión implementada de CG en una librería para hacer una comparación un poco más "justa".

Además, se esperaba que para el caso de las primeras dos funciones cuadráticas el método de CG superara al método de AG pero ocurrió que en la función 1 el método de AG fue más rápido, lo cuál nos pareció extraño considerando que CG es en teoría el método óptimo para la solución de funciones cuadráticas, de nuevo esto podría estar relacionado con nuestra implementación del método de CG.

Sin embargo, consideramos que en general los resultados son bastante positivos, esperabamos que el método del artículo superara en eficiencia a los métodos de CG y AG para corroborar que logra cerrar la brecha entre los dos métodos, y efectivamente lo logró, resultados que en general además concuerdan con los obtenidos por los autores en el artículo.


# Referencias

[1]  Nonlinear conjugate gradient for smooth convex
functions. Sahar Karimi, Stephen Vavasis.

https://arxiv.org/pdf/2111.11613